## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data = pd.read_csv("../Weather_Database/weatherPy_database.csv")
city_data.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,2021-09-23 01:13:28,-53.1500,-70.9167,41.11,70,0,17.27,clear sky
1,1,Rikitea,PF,2021-09-23 01:15:53,-23.1203,-134.9692,74.08,74,0,19.84,light rain
2,2,Airai,TL,2021-09-23 01:15:53,-8.9266,125.4092,72.54,33,44,5.01,scattered clouds
3,3,Castro,BR,2021-09-23 01:15:53,-24.7911,-50.0119,48.06,89,95,9.57,overcast clouds
4,4,Bredasdorp,ZA,2021-09-23 01:15:55,-34.5322,20.0403,58.28,93,0,5.75,clear sky


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 80


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_data = city_data.loc[(city_data["Max Temp"] <= max_temp) & \
                                       (city_data["Max Temp"] >= min_temp)]
preferred_cities_data.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Rikitea,PF,2021-09-23 01:15:53,-23.1203,-134.9692,74.08,74,0,19.84,light rain
2,2,Airai,TL,2021-09-23 01:15:53,-8.9266,125.4092,72.54,33,44,5.01,scattered clouds
6,6,Vaini,TO,2021-09-23 01:15:55,-21.2000,-175.2000,78.96,89,40,10.36,scattered clouds
7,7,Byron Bay,AU,2021-09-23 01:11:34,-28.6500,153.6167,75.06,45,0,5.28,clear sky
14,14,Carnarvon,AU,2021-09-23 01:15:58,-24.8667,113.6333,71.67,73,58,11.50,broken clouds
15,15,Gemena,CD,2021-09-23 01:15:58,3.2500,19.7667,69.44,99,100,2.55,overcast clouds
16,16,Bonito,BR,2021-09-23 01:15:59,-21.1211,-56.4819,78.40,44,100,3.40,overcast clouds
18,18,Meulaboh,ID,2021-09-23 01:10:29,4.1363,96.1285,77.45,85,80,3.67,broken clouds
21,21,Avarua,CK,2021-09-23 01:16:00,-21.2078,-159.7750,75.25,60,20,9.22,few clouds
25,25,Atuona,PF,2021-09-23 01:16:01,-9.8000,-139.0333,78.12,72,0,17.98,clear sky


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_data.isnull().sum()

City_ID                0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_data = preferred_cities_data.dropna()
preferred_cities_data.count()

City_ID                213
City                   213
Country                213
Date                   213
Lat                    213
Lng                    213
Max Temp               213
Humidity               213
Cloudiness             213
Wind Speed             213
Current Description    213
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_data = preferred_cities_data[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_data["Hotel Name"] = ""
hotel_data.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,74.08,light rain,-23.1203,-134.9692,
2,Airai,TL,72.54,scattered clouds,-8.9266,125.4092,
6,Vaini,TO,78.96,scattered clouds,-21.2000,-175.2000,
7,Byron Bay,AU,75.06,clear sky,-28.6500,153.6167,
14,Carnarvon,AU,71.67,broken clouds,-24.8667,113.6333,
15,Gemena,CD,69.44,overcast clouds,3.2500,19.7667,
16,Bonito,BR,78.40,overcast clouds,-21.1211,-56.4819,
18,Meulaboh,ID,77.45,broken clouds,4.1363,96.1285,
21,Avarua,CK,75.25,few clouds,-21.2078,-159.7750,
25,Atuona,PF,78.12,clear sky,-9.8000,-139.0333,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_data.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_data.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print('No hotel found near this location...skipping')

No hotel found near this location...skipping
No hotel found near this location...skipping
No hotel found near this location...skipping
No hotel found near this location...skipping
No hotel found near this location...skipping
No hotel found near this location...skipping
No hotel found near this location...skipping
No hotel found near this location...skipping
No hotel found near this location...skipping
No hotel found near this location...skipping
No hotel found near this location...skipping
No hotel found near this location...skipping
No hotel found near this location...skipping
No hotel found near this location...skipping
No hotel found near this location...skipping
No hotel found near this location...skipping
No hotel found near this location...skipping
No hotel found near this location...skipping
No hotel found near this location...skipping


In [ ]:
# 7. Drop the rows where there is no Hotel Name.
hotel_data = hotel_data.loc[hotel_data["Hotel Name"] != ""]
hotel_data.head(10)

In [ ]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
hotel_data.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_data.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_data[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 

# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Marker layer
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add the map layers.
fig.add_layer(marker_layer)

# 11b. Display the figure
fig
